# Assignment 5
Simulate 1000 battle rounds in Risk.

Undertaken for the [Programming for Data Analytics](https://www.atu.ie/courses/higher-diploma-in-science-data-analytics#:~:text=Programming%20for%20Data%20Analytics) module as part of the [Higher Diploma in Science in Data Analytics](https://www.atu.ie/courses/higher-diploma-in-science-data-analytics) at ATU.

In [234]:
import numpy as np # to utilise random number generation
import matplotlib.pyplot as plt # for plotting

I will attempt to write a function that simulates a single round of Risk: I

In [235]:
# simulate a single round of Risk
def risk_round():
    # no losses at start of round
    defend_loss = 0
    attack_loss = 0
   
    # attacker and defender choose number of troops to put forward
    attack_dice = np.random.randint(1, 4)
    defend_dice = np.random.randint(1, 3)

    # simulating dice rolls for the chosen number of troops
    attack_rolls = [np.random.randint(1, 7) for _ in range(attack_dice)]
    defend_rolls = [np.random.randint(1, 7) for _ in range(defend_dice)]

    # sorting attack and defence in descending order
    sorted_attack = sorted(attack_rolls, reverse=True)
    sorted_defend = sorted(defend_rolls, reverse=True)

    # comparing max attack and max defence dice
    max_attack = max(sorted_attack)
    max_defend = max(sorted_defend)

    if max_attack > max_defend:
        defend_loss -= 1 # if the max attack roll is greater than max defend, one defend loss
    else:
        attack_loss -= 1 # if max attack is equal or less, one attack loss

    # handling if only one dice by either player
    if len(sorted_attack) >1 and len(sorted_defend) >1:
        two_attack = sorted_attack[1]
        two_defend = sorted_defend[1]
       
       # comparing 2nd attack and 2nd defence dice
        if two_attack > two_defend:
            defend_loss -= 1 # if the 2nd attack roll is greater than 2nd defend, one defend loss
        else:
            attack_loss -= 1 # if 2nd attack is equal or less, one attack loss

    else:
        two_attack = "n/a"
        two_defend = "n/a"

    #return attack and defend loss for this round
    return attack_loss, defend_loss

In [236]:
risk_round()

(0, -1)

In [237]:
# Track cumulative losses over 1000 rounds
total_attack_loss = 0
total_defend_loss = 0

# Run the function 1000 times and accumulate the losses
for x in range(1000):
    attack_loss, defend_loss = risk_round()
    total_attack_loss += attack_loss
    total_defend_loss += defend_loss

# Output the total losses after 1000 rounds
print(f"Total attack losses over 1000 rounds: {total_attack_loss}")
print(f"Total defend losses over 1000 rounds: {total_defend_loss}")

Total attack losses over 1000 rounds: -698
Total defend losses over 1000 rounds: -634
